In [1]:
import numpy as np

In [ ]:
class Model:
    def __init__(self, X, y, bounds, regression_function, *, train_split=0.8):
        '''
        Intialize Model class

        X: 1-D array. Features array
        y: 1-D array. Labels array
        bounds: 2-D array. Parameters bound array of (left_bound, right_bound) tuples. left_bound < right_bound
        train_split: float in (0, 1) 
        regression_funcion: function to be fitted. Signature is (y, **params)
        '''
        self.train_test_partition(X, y, train_split)
        self.init_params(bounds)

        self.fit_function = np.vectorize(regression_function, excluded=self.params.keys())

    def train_test_partition(self, X, y, train_split):
        '''
        Splitting data into test and train sets
        '''
        n = int(len(X) * train_split)

        data = zip(X, y)
        shuffled_data = np.random.shuffle(data)
        self.X_train, self.y_train = shuffled_data[:n][0], shuffled_data[:n][1]
        self.X_test, self.y_test = shuffled_data[n:][0], shuffled_data[n:][1]

    def init_params(self, bounds):
        '''
        Intializes parameters
        '''
        self.params = {}
        for i, (left, right) in enumerate(bounds):
            self.params[f"b{i}"] = np.random.rand() * (right - left) + left

    def loss(self):
        '''
        Computes loss function
        '''
        y_hat = self.fit_function(self.X_train)
        mse = (y_hat - self.y_train) ** 2 / 2
        return mse